In [1]:
import pandas as pd
import jieba

import jieba.posseg as pseg
jieba.set_dictionary('jieba/dict.txt')

jieba.load_userdict('userdict/symptom.txt')
jieba.load_userdict('userdict/time.txt')
jieba.load_userdict('userdict/position.txt')
jieba.load_userdict('userdict/others.txt')
jieba.initialize()

import re
from collections import defaultdict
import matplotlib.pyplot as plt

import time as timer
start_time = timer.time()

position_replace = {
    #'疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '腹部':'腹','肚子':'腹',
    '胸部':'胸','胸口':'胸',
    '肩胛骨':'背','背部':'背',
    '臉部':'臉','臉頰':'臉','鼻子':'臉','耳朵':'臉','唇':'臉','眼':'臉','嘴唇':'臉','額頭':'臉',
    '脖子':'頸','頸部':'頸','後頸':'頸',
    '嘴巴':'嘴','牙齦':'嘴','口腔':'嘴','唇':'嘴',
}

sym_replace = {
    '疼痛':'痛','疼':'痛','痠痛':'痛','脹痛':'痛','腫痛':'痛','絞痛':'痛','悶痛':'痛',
    '嘔吐':'吐','想吐':'吐',
    '心跳快':'心悸',
    '麻木':'麻',
    '發冷':'畏寒',
    '疹子':'紅疹',
}

f_replace = {
    '左側':'左',
    '右側':'右'
}

kcom = pd.read_csv('Kcom_dr568410605emgcase.csv', lineterminator='\n')
kcom['Kcom'] = kcom['Kcom'].fillna("並無主訴")

print(len(kcom))
kcom.head()

Building prefix dict from /Users/kuanwen/Desktop/自然語言/project/jieba/dict.txt ...
Dumping model to file cache /var/folders/tv/yy9kh1h15dsfg0vcy7vbgtmw0000gn/T/jieba.u936d01f3f8b35df8fcf902039c509f73.cache
Loading model cost 3.785 seconds.
Prefix dict has been built succesfully.


241822


,Pno,Caseno,Kcom
0,15929545,"5,621",今天下午開始雙腳抽筋 現求治
1,15941825,"5,011",剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
2,15965603,"6,001",據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
3,15966387,"6,801",昨天開始頭暈不適 現未改善
4,15968549,"5,441",昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中


In [2]:
p = pd.DataFrame(columns=['Pno','Caseno'])
kcom_out = pd.concat([pd.DataFrame(columns=['Kcom_'+str(i),'Kcom_du'+str(i)]) for i in range(1,11)], axis=1)
kcom_out = pd.concat([p, kcom_out], axis=1)

kcom_out

,Pno,Caseno,Kcom_1,Kcom_du1,Kcom_2,Kcom_du2,Kcom_3,Kcom_du3,Kcom_4,Kcom_du4,...,Kcom_6,Kcom_du6,Kcom_7,Kcom_du7,Kcom_8,Kcom_du8,Kcom_9,Kcom_du9,Kcom_10,Kcom_du10


In [3]:
max_length = 0

def replace_w(string, replace_dict):
    fin = 0
    
    while fin == 0:
        match = list()
        
        for d in replace_dict:
            if d in string:
                match.append(d)
        
        if len(match) == 0:
            fin = 1
        else:
            max_seg = ''
            for m in match:
                if len(m) > len(max_seg):
                    max_seg = m
            
            string = string.replace(max_seg, replace_dict[max_seg])
    return string
            


with open('output.txt', 'a') as the_file:

    for i in range(len(kcom)):
        split = kcom['Kcom'].loc[i].replace("有","")
        split = split.replace("會","")
        split = split.split()


        t_now = 0
        m = 0
        pre_time = list()
        event_w = list()
        event_now = list()

        for s in split: 
            t_w = list()
            sym_w = list()
            pos_w = list()

            last_t = 0
            last_n = 0
            last_f = 0
            last_sym = 0
            t = 0
            sym = 0



            line = pseg.cut(s,HMM=False)
            for k,f in line:
                print('%s %s' % (k,f), end=' ')

                if f == 't':
                    last_n = 0
                    last_sym = 0
                    last_f = 0
                    if t_now == 0:
                        if sym == 1:
                            event = dict()
                            event['time'] = t_w
                            event['sym'] = sym_w
                            event_w.append(event)
                            t_w = list()
                            sym_w = list()
                            pos_w = list()
                            sym = 0
                            t_w.append(k)
                            last_t = 1
                            t = 1  
                        elif last_t == 1 and t == 1:
                            t_w.append(k)
                            last_t = 1
                            t = 1
                        elif t == 0:
                            t_w.append(k)
                            last_t = 1
                            t = 1

                elif f == 'tnow' or f == 'tg':
                    last_n = 0
                    last_t = 0
                    last_sym = 0
                    last_f = 0
                    t_now = 1
                    if sym == 1:
                        event = dict()
                        event['time'] = t_w
                        event['sym'] = sym_w
                        event_w.append(event)
                        t_w = list()
                        sym_w = list()
                        pos_w = list()
                        sym = 0


                elif f == 'sym':
                    if last_n == 1:
                        for p in pos_w:
                            p = replace_w(p, position_replace)
                            p = replace_w(p, f_replace)
                            k = replace_w(k, sym_replace)
                            sym_w.append(p+k)
                        pos_w = list()
                    else:
                        k = replace_w(k, sym_replace)
                        sym_w.append(k)

                    last_n = 0
                    last_t = 0
                    last_sym = 1
                    last_f = 0
                    sym = 1

                elif f == 'f':
                    last_f = 1
                    last_sym = 0
                    last_n = 0
                    last_t = 0
                    n = k
                elif f == 'pos':
                    if last_f == 1:
                        n = n+k
                        pos_w.append(n)
                    elif last_n == 1:
                        pos_w.append(k)
                    else: 
                        n = k
                        pos_w.append(n)

                    last_n = 1
                    last_sym = 0
                    last_f = 0
                    last_t = 0
                elif f == 'm' and last_sym == 1:
                    last_t = 0
                    last_f = 0
                    last_n = 0
                    last_sym = 0
                    if len(t_w) == 0:
                        t_w.append(k)
                        m = 1
                        last_t = 1

                else:
                    last_f = 0
                    last_t = 0
                    last_n = 0
                    last_sym = 0

            if len(sym_w) > 0:
                if t_now == 1:
                    event_now += sym_w
                else:
                    event = dict()
                    if len(t_w) == 0:
                        event['time'] = pre_time
                    else:
                        event['time'] = t_w 
                    event['sym'] = sym_w
                    event_w.append(event)
            elif len(t_w) > 0:
                    pre_time = t_w

        total = list()
        
        time = ''
        for e in event_w:
            event = dict()
            #print(e)
            if len(e['time']) > 0:
                time = ''.join(e['time'])
            event['time'] = time
            event['sym'] = e['sym']

            total.append(event)
        
        if m == 1:
            m_t = ''
            for idx, e in reversed(list(enumerate(total))):
                if len(e['time']) > 0:
                    m_t = e['time']
                else:
                    e['time'] = m_t
        
        for idx, e in reversed(list(enumerate(total))):
            if idx != 0:
                if e['time'] == total[idx-1]['time']:
                    #print(e)
                    total[idx-1]['sym'] += e['sym']
                    del total[idx]
        
        
    
        
        if len(event_now) > 0:
            total.append({'time': '現在', 'sym': event_now})
            #for e_now in event_now:
                #total.append({'time': '現在', 'sym': [e_now]})

        '''
        if len(total) > max_length:
            max_length = len(total)
        '''
        
        
        '''
        p = pd.DataFrame(columns=['Pno','Caseno'])
        temp = pd.concat([pd.DataFrame(columns=['Kcom_'+str(t),'Kcom_du'+str(t)]) for t in range(1,11)], axis=1)
        temp = pd.concat([p, temp], axis=1)
        
        temp['Pno'] = [kcom['Pno'].loc[i]]
        temp['Caseno'] = [kcom['Caseno'].loc[i]]
        
        t_idx = 1 
        for period in total:
            #print(period)
            
            if len(period['sym']) > 1:
                for s in period['sym']:
                    temp['Kcom_'+ str(t_idx)] = [s]
                    temp['Kcom_du'+ str(t_idx)] = period['time']
                    t_idx += 1
            else:
                temp['Kcom_'+ str(t_idx)] = period['sym']
                temp['Kcom_du'+ str(t_idx)] = period['time']
                t_idx += 1
        
        kcom_out = kcom_out.append(temp)
        '''
        
        print('%d. %s' % (i+1,kcom['Kcom'].loc[i]))
        #the_file.write('%d. %s\n' % (i+1,kcom['Kcom'].loc[i]))
        for period in total:
            print('時間： %s' % period['time'])
            print('症狀： %s' % ','.join(period['sym']))
            #the_file.write('時間： %s\n' % period['time'])
            #the_file.write('症狀： %s\n' % ','.join(period['sym']))
        print('\n')
        #the_file.write('\n')
    #print(max_length)
    #kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")
    

今天下午 t 開始 v 雙腳 pos 抽筋 sym 現 tnow 求治 v 1. 今天下午開始雙腳抽筋 現求治
時間： 今天下午
症狀： 雙腳抽筋


剛剛 t 在家 r 全身不適 sym 臉潮紅 sym 在家 r 量 v 血壓高 sym 現 tnow 求治 v 2. 剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
時間： 剛剛
症狀： 全身不適,臉潮紅,血壓高


據 p 家屬 n 訴 vn 今天 t 早上 t 開始 v 食慾差 sym 不說話 sym 全身 pos 無力 sym 現 tnow 意識不清 sym 119 eng 測 zg 血糖 n 33MG eng / x DL eng 3. 據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
時間： 今天早上
症狀： 食慾差,不說話,全身無力
時間： 現在
症狀： 意識不清


昨天 t 開始 v 頭暈 sym 不適 sym 現 tnow 未 d 改善 v 4. 昨天開始頭暈不適 現未改善
時間： 昨天
症狀： 頭暈,不適


昨天 t 開始 v 左 f 腰 pos 左 f 腹 pos 痛 sym 想吐 sym 外院 n 檢查 vn 結石 n 現 tnow 持續 vd 中 n 5. 昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中
時間： 昨天
症狀： 左腹痛,吐


腹 pos 痛 sym 兩天 m 現 tnow 求治 v 6. 腹痛兩天 現求治
時間： 兩天
症狀： 腹痛


昨天晚上 t 開始 v 腹 pos 痛 sym 想吐 sym LMP eng ： x 8 eng / x 5 eng 7. 昨天晚上開始腹痛 想吐 LMP：8/5
時間： 昨天晚上
症狀： 腹痛,吐


剛剛 t 睡覺 v 忽然 d 開始 v 頭 pos 痛 sym 不適 sym 8. 剛剛睡覺忽然開始頭痛不適
時間： 剛剛
症狀： 頭痛,不適


昨天晚上 t 開始 v 腹 pos 痛 sym 不適 sym 現 tnow 求治 v 9. 昨天晚上開始腹痛不適 現求治
時間： 昨天晚上
症狀： 腹痛,不適


今天 t 早上 t 開始 v 發燒 sym 喉嚨 pos 痛 sym 咳嗽 sym 肌肉 pos 痠痛 sym 外院 n 求治 v 過 ug 現 tnow 未 d 改善 v 10. 今天早上開

週二 t 開始 v 發燒 sym 畏寒 sym 全身 pos 痠痛 sym 現 tnow 求治 v 145. 週二開始發燒 畏寒 全身痠痛 現求治
時間： 週二
症狀： 發燒,畏寒,全身痛


家屬 n 代 q 訴 vn 剛剛 t 開始 v 發燒 sym 現 tnow 求治 v 146. 家屬代訴剛剛開始發燒  現求治
時間： 剛剛
症狀： 發燒


剛開始 t 左 f 腰 pos 痛 sym 利害 v 現 tnow 求治 v 147. 剛開始左腰痛利害現求治
時間： 剛開始
症狀： 左腰痛


剛開始 t 頭暈 sym 腹 pos 痛 sym 現 tnow 求治 v 148. 剛開始頭暈  腹痛  現求治
時間： 剛開始
症狀： 頭暈,腹痛


全身 pos 無力 sym 多天 m 現 tnow 未 d 改善 v 149. 全身無力多天 現未改善
時間： 多天
症狀： 全身無力


昨天 t 開始 v 右 f 腰 pos 痛 sym 現 tnow 未 d 改善 v 150. 昨天開始右腰痛 現未改善
時間： 昨天
症狀： 右腰痛


剛開始 t 胸 pos 痛 sym 現 tnow 求治 v 151. 剛開始胸痛  現求治
時間： 剛開始
症狀： 胸痛


今天 t 晚上 t 開始 v 上吐下瀉 sym 腹 pos 痛 sym 152. 今天晚上開始上吐下瀉 腹痛
時間： 今天晚上
症狀： 上吐下瀉,腹痛


今早 t 開始 v 發燒 sym 全身 pos 酸痛 sym 現 tnow 求治 v 8 eng / x 13 eng - x 8 eng / x 16 eng 香港 ns 洽 zg 公 n 153. 今早開始發燒 全身酸痛 現求治  8/13-8/16香港洽公
時間： 今早
症狀： 發燒,全身酸痛


剛開始 t 頭暈 sym 現 tnow 求治 v 154. 剛開始頭暈  現求治
時間： 剛開始
症狀： 頭暈


星期三 t 開始 v 嘔吐 sym 腹瀉 sym 頭暈 sym 現 tnow 求治 v 155. 星期三開始嘔吐 腹瀉  頭暈  現求治
時間： 星期三
症狀： 吐,腹瀉,頭暈


昨天 t 開始 v 喘 sym SPO2 eng ： x 96 eng ％ x 雙腳 pos 麻 sym 現 tnow 求治 v 156. 昨天開始喘 SPO2：9


左 f 腹 pos 痛 sym 便秘 sym 2 eng - x 3天 m 自行 r 買 v 瀉藥 n 解 v 出 v 現 tnow 疼痛 sym 仍 zg 未 d 改善 v LMP eng : x 8 eng 月初 t 264. 左腹痛 便秘2-3天 自行買瀉藥有解出  現疼痛仍未改善  LMP:8月初
時間： 
症狀： 左腹痛,便秘
時間： 現在
症狀： 痛


下午 t 開始 v 胃 pos 痛 sym 想吐 sym LMP eng 上週 t 一 m 265. 下午開始 胃痛 想吐 LMP上週一
時間： 下午
症狀： 胃痛,吐


今天 t 晚上 t 開始 v 全身 pos 起紅疹 sym 癢 sym 現 tnow 求治 v 266. 今天晚上開始全身起紅疹癢 現求治
時間： 今天晚上
症狀： 全身起紅疹,癢


今天下午 t 四點 m 開始 v 腹瀉 sym 現 tnow 未 d 改善 v 267. 今天下午四點開始腹瀉 現未改善
時間： 今天下午
症狀： 腹瀉


走路 n 不 d 穩 a 多天 m 暈 sym 現 tnow 未 d 改善 v 268. 走路不穩多天 會暈 現未改善
時間： 
症狀： 暈


今早 t 開始 v 腹瀉 sym 胃 pos 痛 sym 解 v 尿 n 疑似 v 血 n 269. 今早開始腹瀉 胃痛 解尿疑似有血
時間： 今早
症狀： 腹瀉,胃痛


剛 t 吃安眠藥 sym 喝完 v 酒後 t 開始 v 嘔吐 sym 現 tnow 未 d 改善 v 270. 剛吃安眠藥 喝完酒後開始嘔吐 現未改善
時間： 剛
症狀： 吃安眠藥
時間： 酒後
症狀： 吐


今天 t 早上 t 睡醒 v 後 n 開始 v 頭暈 sym 左腳 pos 無力 sym 門診 n 轉入 v 待 v 住院 n 271. 今天早上睡醒後開始頭暈 左腳無力 門診轉入待住院
時間： 今天早上
症狀： 頭暈,左腳無力


頭暈 sym 多天 m 現 tnow 嘔吐 sym 血壓高 sym 求治 v 272. 頭暈多天  現嘔吐  血壓高求治
時間： 多天
症狀： 頭暈
時間： 現在
症狀： 吐,血壓高


今天下午 t 開始 v 胃 pos 痛 sym 現 tnow 未 d 改善 v 273. 今天下午開始胃痛 現未改善
時間： 今天下午
症狀： 胃痛


早

昨天 t 開始 v 全身 pos 紅疹 sym 搔癢 sym 求治 v 401. 昨天開始全身紅疹 搔癢求治
時間： 昨天
症狀： 全身紅疹,搔癢


今天 t 早上 t 開始 v 左側 f 胸口 pos 下方 n 疼痛 sym 外院 n 診所 v 懷疑 v 氣胸 n 402. 今天早上開始 左側胸口下方疼痛 外院診所懷疑氣胸
時間： 今天早上
症狀： 痛


胃 pos 痛 sym 多天 m 昨天 t 開始 v 上吐下瀉 sym 現 tnow 求治 v 403. 胃痛多天 昨天開始上吐下瀉 現求治
時間： 多天
症狀： 胃痛
時間： 昨天
症狀： 上吐下瀉


今天 t 晚上 t 開始 v 頭暈 sym 旋轉 v 感 v 現 tnow 求治 v 404. 今天晚上開始頭暈會有旋轉感 現求治
時間： 今天晚上
症狀： 頭暈


今天 t 晚上 t 開始 v 頭暈 sym 嘔吐 sym 405. 今天晚上開始頭暈 嘔吐
時間： 今天晚上
症狀： 頭暈,吐


剛剛 t 頭暈 sym 跌倒 v 現 tnow 左 f 唇 pos 傷口 pos 右膝 pos 傷口 pos 救護車 n 血糖 n 504mg eng / x d eng 406. 剛剛頭暈跌倒 現左唇有傷口 右膝有傷口 救護車血糖504mg/d
時間： 剛剛
症狀： 頭暈


要 v 聯絡 n 洗 v 腎 pos 407. 要聯絡洗腎


前天 t 開始 v 腹 pos 痛 sym 腹瀉 sym 現 tnow 求治 v 408. 前天開始腹痛  腹瀉 現求治
時間： 前天
症狀： 腹痛,腹瀉


今天 t 中午 t 開始 v 發燒 sym 門診 n 求治 v 建議 n 轉 zg 急診 v 待 v 床 n 409. 今天中午開始發燒 門診求治 建議轉急診待床
時間： 今天中午
症狀： 發燒


中午 t 11 eng 點 zg 開始 v 嘴巴 pos 歪 v 臉部 pos 腫 sym 現 tnow 求治 v 410. 中午11點開始嘴巴歪 臉部腫 現求治
時間： 中午
症狀： 臉腫


2天前 t 開始 v 腹瀉 sym 現 tnow 求治 v 411. 2天前開始腹瀉  現求治
時間： 2天前
症狀： 腹瀉


前天 t 開始 v 咳嗽 sym 喉嚨 pos 痛 sym 現 tnow 求治 v 兒子 n 為 z

剛剛 t 開始 v 胃 pos 痛 sym 想吐 sym 頭 pos 痛 sym 現 tnow 求治 v lmp7 eng 月底 t 服用 vn 一顆 m 安眠藥 n 545. 剛剛開始胃痛 想吐頭痛  現求治lmp7月底 有服用一顆安眠藥
時間： 剛剛
症狀： 胃痛,吐,頭痛


家屬 n 訴 vn 昨天 t 開始 v 血壓高 sym 吸吸 v 喘 sym SPO2 eng 98 eng % x 119 eng 車上 s 驗 v 血糖 n - x - x 449mg eng / x dl eng 546. 家屬訴  昨天開始血壓高 吸吸喘  SPO2 98% 119車上驗血糖--449mg /dl
時間： 昨天
症狀： 血壓高,喘


剛剛 t 上班 v 中 n 吃 v 完 v 口服藥 n 後 n 呼吸喘 sym 雙手 pos 抖 zg 現 tnow 求治 v SPO2 eng 100 eng 547. 剛剛上班中吃完口服藥後呼吸喘 雙手抖 現求治 SPO2 100
時間： 剛剛
症狀： 呼吸喘


今天 t 開始 v 腹 pos 痛 sym 現 tnow 求治 v 548. 今天開始腹痛 現求治
時間： 今天
症狀： 腹痛


今天 t 晚上 t 七點 m 開始 v 上 n 腹 pos 及 c 胸口 pos 疼痛 sym 現 tnow 求治 v 549. 今天晚上七點開始上腹及胸口疼痛 現求治
時間： 今天晚上
症狀： 腹痛,胸痛


發燒 sym 3天 m 全身 pos 無力 sym 肌肉 pos 痠痛 sym 腹瀉 sym 腹 pos 痛 sym 腹脹 sym 吃不下 sym 現 tnow 求治 v 550. 發燒3天 全身無力 肌肉痠痛 腹瀉 腹痛 腹脹 吃不下 現求治
時間： 3天
症狀： 發燒,全身無力,肌肉痛,腹瀉,腹痛,腹脹,吃不下


昨天 t 開始 v 腹瀉 sym 想吐 sym 頭暈 sym 現 tnow 求治 v LMP eng : x 7 eng / x 24 eng 551. 昨天開始腹瀉 想吐 頭暈  現求治 LMP:7/24
時間： 昨天
症狀： 腹瀉,吐,頭暈


據 p 自殺 sym 防治 v 中心 n 人員 n 代 q 訴 vn 剛 t 在 p 中心 n 門口 s 突然 ad 情緒激動 sym 躁動 vn 無法控制 nz 55

...

時間： 昨天晚上
症狀： 頻尿,痛,左腰痛


胃 pos 痛 sym 多天 m LMD eng 就診 v 過 ug 現 tnow 仍 zg 不適 sym 3157. 胃痛多天 LMD就診過 現仍不適
時間： 多天
症狀： 胃痛
時間： 現在
症狀： 不適


昨天 t 開始 v 腹 pos 痛 sym 嘔吐 sym 現 tnow 仍 zg 腹 pos 痛 sym 求治 v 已 d 停經 v 3158. 昨天開始腹痛嘔吐 現仍腹痛求治 已停經
時間： 昨天
症狀： 腹痛,吐
時間： 現在
症狀： 腹痛


今天 t 早上 t 開始 v 頭暈 sym 想吐 sym 現 tnow 求治 v 3159. 今天早上開始頭暈 想吐 現求治
時間： 今天早上
症狀： 頭暈,吐


剛 t 一 m 小時 n 前 n 開始 v 上 n 腹 pos 悶痛 sym 胸 pos 痛痛 a 到 v 背後 f 冒冷汗 sym 不適 sym 3160. 剛一小時前開始上腹悶痛 胸痛痛到背後 冒冷汗不適
時間： 剛
症狀： 腹痛,冒冷汗,不適


凌晨 t 腹 pos 痛 sym 現 tnow 未 d 改善 v LMP eng : x 1 eng 月 m 3161. 凌晨腹痛 現未改善 LMP:1月
時間： 凌晨
症狀： 腹痛


解尿痛 sym 解 v 血尿 sym 多天 m 現 tnow 未 d 改善 v 3162. 解尿痛 解血尿多天 現未改善
時間： 多天
症狀： 解尿痛,血尿


今天 t 早上 t 開始 v 頭 pos 痛 sym 嘔吐 sym 現 tnow 求治 v 月經 n 第二天 m 3163. 今天早上開始頭痛 有嘔吐 現求治 月經第二天
時間： 今天早上
症狀： 頭痛,吐


周一 t 開始 v 頭 pos 痛 sym 3164. 周一開始頭痛
時間： 周一
症狀： 頭痛


頭 pos 痛 sym 多天 m 現 tnow 發燒 sym 中 n 求治 v 3165. 頭痛多天 現發燒中求治
時間： 多天
症狀： 頭痛
時間： 現在
症狀： 發燒


頭暈 sym 一 m 星期 n 診所 v 求治 v 過 ug 現 tnow 仍 zg 不適 sym 3166. 頭暈一星期 診所求治過 現仍不適
時間： 一
症狀： 頭暈
時間： 現在
症狀： 不適


呼吸喘 sym 多天 m

症狀： 肺炎


今天 t 凌晨 t 開始 v 胃 pos 痛 sym 不適 sym 現 tnow 持續 vd 中 n 3302. 今天凌晨開始胃痛不適 現持續中
時間： 今天凌晨
症狀： 胃痛,不適


最近 n 覺 v 坐立不安 i 胸悶 sym 失眠 sym 3303. 最近覺坐立不安   胸悶   失眠
時間： 
症狀： 胸悶,失眠


今天 t 開始 v 頭暈 sym 站不穩 l 3304. 今天開始頭暈  站不穩
時間： 今天
症狀： 頭暈


家屬 n 代 q 訴 vn 剛剛 t 自覺 d 快 a 昏倒 sym 全身 pos 無力 sym 現 tnow 求治 v 3305. 家屬代訴剛剛自覺快昏倒 全身無力  現求治
時間： 剛剛
症狀： 昏倒,全身無力


剛開始 t 腹瀉 sym 腹 pos 痛 sym lmp2 eng / x 27 eng 現 tnow 求治 v 3306. 剛開始腹瀉 腹痛lmp2/27 現求治
時間： 剛開始
症狀： 腹瀉,腹痛


昨天晚上 t 開始 v 胃 pos 痛 sym 發燒 sym 現 tnow 求治 v 月經 n 第 m 1天 m 3307. 昨天晚上開始胃痛 發燒 現求治 月經第1天
時間： 昨天晚上
症狀： 胃痛,發燒


今天 t 小便 pos 疼痛 sym 解 v 血尿 sym 現 tnow 求治 v 3308. 今天小便疼痛 解血尿 現求治
時間： 今天
症狀： 小便痛,血尿


咳嗽 sym 多天 m 現 tnow 頭暈 sym 食欲 n 差 a 求治 v 3309. 咳嗽多天 現頭暈 食欲差 求治
時間： 多天
症狀： 咳嗽
時間： 現在
症狀： 頭暈


今天 t 早上 t 開始 v 頭暈 sym 嘔吐 sym 現 tnow 求治 v 3310. 今天早上開始頭暈 嘔吐 現求治
時間： 今天早上
症狀： 頭暈,吐


昨天 t 開始 v 頭 pos 痛 sym 現 tnow 持續 vd 中 n 3311. 昨天開始頭痛 現持續中
時間： 昨天
症狀： 頭痛


今晚 t 開始 v 頻尿 sym 小便 pos 疼痛 sym 雙 f 側 v 腰痠 sym 現 tnow 求治 v 3312. 今晚開始頻尿 小便疼痛 雙側腰痠 現求治
時間： 今晚
症狀： 頻尿,小便痛,腰痠


咳嗽 sym 痰 sy

症狀： 全身痛,咳嗽,流鼻水,喉嚨痛


晚上 t 開始 v 解 v 血尿 sym 頻尿 sym 頻尿 sym 現 tnow 求治 v LMP3 eng / x 30 eng 3452. 晚上開始解血尿 頻尿 頻尿 現求治 LMP3/30
時間： 晚上
症狀： 血尿,頻尿,頻尿


腹 pos 痛 sym 兩天 m 現 tnow 求治 v 3453. 腹痛兩天 現求治
時間： 兩天
症狀： 腹痛


剛 t 在 p 路上 s 突然 ad 胸 pos 痛 sym 冒冷汗 sym 直 d 入 v 3454. 剛在路上突然胸痛冒冷汗直入
時間： 剛
症狀： 胸痛,冒冷汗


昨天晚上 t 開始 v 左 f 腰 pos 痛 sym 現 tnow 仍 zg 不適 sym 3455. 昨天晚上開始左腰痛 現仍不適
時間： 昨天晚上
症狀： 左腰痛
時間： 現在
症狀： 不適


左 f 腰 pos 痛 sym 2天 m 現 tnow 仍 zg 不適 sym 3456. 左腰痛2天 現仍不適
時間： 2天
症狀： 左腰痛
時間： 現在
症狀： 不適


剛剛 t 開始 v 右 f 上 n 腹 pos 痛 sym 3 eng / x 23 eng ~ x 3 eng / x 31 eng 大陸 n 3457. 剛剛開始右上腹痛 3/23~3/31大陸
時間： 剛剛
症狀： 腹痛


剛開始 t 嘔吐 sym 在家 r 量 v 血壓高 sym 現 tnow 求治 v 3458. 剛開始嘔吐 在家量血壓高 現求治
時間： 剛開始
症狀： 吐,血壓高


剛開始 t 頭暈 sym 不適 sym 量 v 血壓高 sym 高血壓 sym 3459. 剛開始頭暈不適 量血壓高高血壓
時間： 剛開始
症狀： 頭暈,不適,血壓高,高血壓


腹脹 sym 一個月 m 現 tnow 求治 v 3460. 腹脹一個月 現求治
時間： 一個月
症狀： 腹脹


多天 m 開始 v 腹脹 sym 胃 pos 痛 sym 想吐 sym OPD eng 求治 v 建議 n 來 zg 抽 v 腹水 n 3461. 多天開始腹脹 胃痛 想吐  OPD求治建議來抽腹水
時間： 
症狀： 腹脹,胃痛,吐


今天 t 晚上 t 6 eng 點 zg 開始 v 上吐下瀉 sym LMP eng : x 3 eng / 

昨天晚上 t 開始 v 左 f 腳踝 pos 痛 sym 看過 v 醫生 n 現 tnow 仍 zg 痛 sym 3602. 昨天晚上開始左腳踝痛 有看過醫生 現仍痛
時間： 昨天晚上
症狀： 左腳踝痛
時間： 現在
症狀： 痛


剛剛 t 開始 v 左 f 腰 pos 痠痛 sym 現 tnow 求治 v 3603. 剛剛開始左腰痠痛 現求治
時間： 剛剛
症狀： 左腰痛


剛剛 t 開始 v 腹 pos 痛 sym 腹瀉 sym 3604. 剛剛開始腹痛 腹瀉
時間： 剛剛
症狀： 腹痛,腹瀉


今天 t 開始 v 腹 pos 痛 sym 3605. 今天開始腹痛
時間： 今天
症狀： 腹痛


今天 t 早上 t 開始 v 胃 pos 痛 sym 想吐 sym 頭暈 sym 現 tnow 求治 v 3606. 今天早上開始胃痛 想吐 頭暈 現求治
時間： 今天早上
症狀： 胃痛,吐,頭暈


早上 t 開始 v 腹 pos 痛 sym 腹脹 sym 3607. 早上開始腹痛腹脹
時間： 早上
症狀： 腹痛,腹脹


2天 m 未 d 解 v 便 d 現 tnow 腹 pos 痛 sym 腹脹 sym 3608. 2天未解便 現腹痛 腹脹
時間： 現在
症狀： 腹痛,腹脹


今天下午 t 開始 v 左 f 腰 pos 痛 sym 厲害 a 現 tnow 求治 v 3609. 今天下午開始左腰痛厲害 現求治
時間： 今天下午
症狀： 左腰痛


今天下午 t 開始 v 解尿困難 sym 現 tnow 血壓高 sym 求治 v 3610. 今天下午開始解尿困難 現血壓高求治
時間： 今天下午
症狀： 解尿困難
時間： 現在
症狀： 血壓高


解尿困難 sym 多天 m 急診 v 求治 v 未 d 改善 v 現 tnow 血壓高 sym 求治 v 外 f ON eng FOLEY eng 4 eng / x 18 eng 3611. 解尿困難多天 急診求治未改善 現血壓高求治 外ON FOLEY 4/18
時間： 多天
症狀： 解尿困難
時間： 現在
症狀： 血壓高


解尿困難 sym 多天 m 剛剛 t 來 zg 急診 v 求治 v 現 tnow 仍 zg 滲 v 尿 n 現 tnow 腹 pos 痛 sym 腹脹 sym 血壓高 sym 求治 v 外 

KeyboardInterrupt: 

In [ ]:
#kcom_out.to_csv('kcom_output_0526.csv', index=False,encoding="utf_8_sig")
#kcom_out.head()

In [ ]:
#import time
print("--- %s seconds ---" % (timer.time() - start_time))

In [ ]:
#kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")